In [1]:
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

In [3]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

# Load your training dataset
train_df = pd.read_csv('new1_cyber_train.csv')
# Load your testing dataset
test_df = pd.read_csv('new1_cyber_test.csv')


In [5]:
# Data Preprocessing
# Find the maximum number of unique words in the dataset
max_words = 100

# Find the maximum length of messages in the dataset
max_len = 50


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text


# Preprocessing for training data
train_df['text'] = train_df['text'].apply(preprocess_text)

# Split the training dataset
X_train, y_train = train_df['text'], train_df['label']

# Tokenization
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Sequence and Padding for training data
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')

# Preprocessing for testing data
test_df['text'] = test_df['text'].apply(preprocess_text)

# Split the testing dataset
X_test, y_test = test_df['text'], test_df['label']

# Sequence and Padding for testing data
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')




## Building and Training Models

## DNN

In [6]:
# dnn_model_modified = Sequential([
#     Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
#     Flatten(),
#     Dense(32, activation='sigmoid'),
#     Dropout(0.2),
#     Dense(16, activation='relu'),
#     Dropout(0.7),  # Increased dropout rate
#     Dense(1, activation='sigmoid')
# ])

def create_dnn_model(optimizer='adam',learning_rate=0.001):
    dnn_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Flatten(),
        Dense(32, activation='sigmoid'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.7),  # Increased dropout rate
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    dnn_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

# Wrap the Keras model in a scikit-learn compatible classifier
dnn_classifier = KerasClassifier(build_fn=create_dnn_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_dnn = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_dnn = GridSearchCV(estimator=dnn_classifier, param_grid=param_grid_dnn, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_dnn = grid_search_dnn.fit(X_train_padded, y_train)


C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/30


19/19 - 1s - loss: 0.7778 - accuracy: 0.4942 - 1s/epoch - 61ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7257 - accuracy: 0.4925 - 27ms/epoch - 1ms/step
Epoch 3/30
19/19 - 0s - loss: 0.6951 - accuracy: 0.5341 - 28ms/epoch - 1ms/step
Epoch 4/30
19/19 - 0s - loss: 0.7020 - accuracy: 0.5008 - 25ms/epoch - 1ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6991 - accuracy: 0.4925 - 24ms/epoch - 1ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6922 - accuracy: 0.5275 - 25ms/epoch - 1ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6954 - accuracy: 0.4792 - 27ms/epoch - 1ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6933 - accuracy: 0.5324 - 33ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6936 - accuracy: 0.5008 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6971 - accuracy: 0.4975 - 33ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6923 - accuracy: 0.5291 - 30ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6973 - accuracy: 0.4942 - 30ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 1s - loss: 0.8287 - accuracy: 0.4925 - 876ms/epoch - 46ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7307 - accuracy: 0.5058 - 35ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7082 - accuracy: 0.4725 - 31ms/epoch - 2ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6968 - accuracy: 0.4992 - 34ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6908 - accuracy: 0.5258 - 35ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6917 - accuracy: 0.5042 - 33ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6955 - accuracy: 0.4825 - 33ms/epoch - 2ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6902 - accuracy: 0.5225 - 33ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6905 - accuracy: 0.5308 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6908 - accuracy: 0.5258 - 33ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6939 - accuracy: 0.4975 - 29ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6896 - accuracy: 0.5125 - 31ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6895 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

19/19 - 1s - loss: 0.7451 - accuracy: 0.4809 - 784ms/epoch - 41ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7118 - accuracy: 0.5125 - 29ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7094 - accuracy: 0.5008 - 28ms/epoch - 1ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6926 - accuracy: 0.4958 - 29ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.7022 - accuracy: 0.5025 - 29ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6899 - accuracy: 0.5141 - 29ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6889 - accuracy: 0.5358 - 28ms/epoch - 1ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6940 - accuracy: 0.5291 - 31ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6877 - accuracy: 0.5308 - 30ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6853 - accuracy: 0.5607 - 30ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6869 - accuracy: 0.5408 - 30ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6905 - accuracy: 0.5308 - 29ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6874 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 1s - loss: 0.7599 - accuracy: 0.5233 - 772ms/epoch - 41ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7272 - accuracy: 0.4850 - 29ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7123 - accuracy: 0.4767 - 30ms/epoch - 2ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6964 - accuracy: 0.5266 - 35ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6962 - accuracy: 0.5050 - 29ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6965 - accuracy: 0.5017 - 31ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6979 - accuracy: 0.4784 - 29ms/epoch - 2ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6909 - accuracy: 0.5083 - 30ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6940 - accuracy: 0.4618 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6932 - accuracy: 0.5183 - 29ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6898 - accuracy: 0.5199 - 29ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6984 - accuracy: 0.4884 - 30ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6925 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


29/29 - 1s - loss: 0.8094 - accuracy: 0.5044 - 769ms/epoch - 27ms/step
Epoch 2/30
29/29 - 0s - loss: 0.7247 - accuracy: 0.5089 - 41ms/epoch - 1ms/step
Epoch 3/30
29/29 - 0s - loss: 0.7084 - accuracy: 0.4701 - 43ms/epoch - 1ms/step
Epoch 4/30
29/29 - 0s - loss: 0.7008 - accuracy: 0.4834 - 42ms/epoch - 1ms/step
Epoch 5/30
29/29 - 0s - loss: 0.6963 - accuracy: 0.4800 - 42ms/epoch - 1ms/step
Epoch 6/30
29/29 - 0s - loss: 0.6969 - accuracy: 0.4867 - 41ms/epoch - 1ms/step
Epoch 7/30
29/29 - 0s - loss: 0.6964 - accuracy: 0.5000 - 43ms/epoch - 1ms/step
Epoch 8/30
29/29 - 0s - loss: 0.6951 - accuracy: 0.4945 - 50ms/epoch - 2ms/step
Epoch 9/30
29/29 - 0s - loss: 0.6911 - accuracy: 0.5200 - 52ms/epoch - 2ms/step
Epoch 10/30
29/29 - 0s - loss: 0.6910 - accuracy: 0.5100 - 51ms/epoch - 2ms/step
Epoch 11/30
29/29 - 0s - loss: 0.6930 - accuracy: 0.5000 - 45ms/epoch - 2ms/step
Epoch 12/30
29/29 - 0s - loss: 0.6865 - accuracy: 0.5266 - 43ms/epoch - 1ms/step
Epoch 13/30
29/29 - 0s - loss: 0.6737 - accura

In [7]:
# Display the results in a table
results_df_dnn = pd.DataFrame(grid_result_dnn.cv_results_)
results_df_dnn[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.743850,9
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.753865,5
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.764939,1
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.716216,14
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.666382,15
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.736109,10
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.763846,3
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.729439,12
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.752743,6
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.734969,11


In [8]:
# Print the best parameters and corresponding accuracy
print("Best Parameters (DNN): ", grid_result_dnn.best_params_)
print("Best Accuracy (DNN): ", grid_result_dnn.best_score_)

Best Parameters (DNN):  {'learning_rate': 0.01, 'optimizer': 'Adagrad'}
Best Accuracy (DNN):  0.7649390919158362


In [20]:
best_dnn_model = grid_result_dnn.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_dnn_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

8/8 - 0s - 49ms/epoch - 6ms/step
Test Accuracy: 0.8008849557522124
Precision: 0.75
Recall: 0.9026548672566371
F1-score: 0.8192771084337348


## LSTM

In [10]:
# lstm_model = Sequential([
#     Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
#     LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
#     Dropout(0.5),
#     LSTM(64),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.3),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])


def create_lstm_model(optimizer='adam', learning_rate=0.001):
    lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
        LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
        Dropout(0.5),
        LSTM(64),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_lstm = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_lstm = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid_lstm, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_lstm = grid_search_lstm.fit(X_train_padded, y_train)


C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 5s - loss: 0.6929 - accuracy: 0.4925 - 5s/epoch - 240ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6941 - accuracy: 0.4992 - 579ms/epoch - 30ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6938 - accuracy: 0.5025 - 569ms/epoch - 30ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6936 - accuracy: 0.5058 - 582ms/epoch - 31ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.4925 - 572ms/epoch - 30ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.5158 - 568ms/epoch - 30ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6939 - accuracy: 0.4925 - 575ms/epoch - 30ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6930 - accuracy: 0.4942 - 566ms/epoch - 30ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6934 - accuracy: 0.5208 - 562ms/epoch - 30ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.4975 - 571ms/epoch - 30ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.5092 - 568ms/epoch - 30ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6922 - accuracy: 0.4958 - 571ms/epoch - 30ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 4s - loss: 0.6938 - accuracy: 0.4908 - 4s/epoch - 215ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6937 - accuracy: 0.5158 - 559ms/epoch - 29ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6938 - accuracy: 0.5042 - 555ms/epoch - 29ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6929 - accuracy: 0.5092 - 546ms/epoch - 29ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.5125 - 560ms/epoch - 29ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.4958 - 545ms/epoch - 29ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6930 - accuracy: 0.5025 - 546ms/epoch - 29ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6931 - accuracy: 0.5175 - 546ms/epoch - 29ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6902 - accuracy: 0.4908 - 548ms/epoch - 29ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6850 - accuracy: 0.4775 - 557ms/epoch - 29ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6805 - accuracy: 0.5208 - 553ms/epoch - 29ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6813 - accuracy: 0.5108 - 556ms/epoch - 29ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
19/19 - 7s - loss: 0.6940 - accuracy: 0.4908 - 7s/epoch - 388ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.5025 - 1s/epoch - 68ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6925 - accuracy: 0.5141 - 1s/epoch - 67ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6937 - accuracy: 0.4892 - 1s/epoch - 69ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.5075 - 1s/epoch - 69ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6928 - accuracy: 0.5125 - 1s/epoch - 68ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6923 - accuracy: 0.5291 - 1s/epoch - 69ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6951 - accuracy: 0.5025 - 1s/epoch - 69ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6935 - accuracy: 0.4892 - 1s/epoch - 67ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6928 - accuracy: 0.5108 - 1s/epoch - 66ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6914 - accuracy: 0.5308 - 1s/epoch - 67ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6912 - accuracy: 0.5075 - 1s/epoch - 68ms/step
Epoch 13/30
19/19 - 1s - loss: 0.7407 - accuracy

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 8s - loss: 0.6927 - accuracy: 0.5100 - 8s/epoch - 408ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6940 - accuracy: 0.5083 - 1s/epoch - 67ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6951 - accuracy: 0.4585 - 1s/epoch - 65ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6934 - accuracy: 0.5116 - 1s/epoch - 67ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6950 - accuracy: 0.4668 - 1s/epoch - 66ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6930 - accuracy: 0.5066 - 1s/epoch - 68ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6938 - accuracy: 0.4934 - 1s/epoch - 66ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6942 - accuracy: 0.4884 - 1s/epoch - 66ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6927 - accuracy: 0.5249 - 1s/epoch - 65ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6947 - accuracy: 0.4801 - 1s/epoch - 65ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6931 - accuracy: 0.5249 - 1s/epoch - 65ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6930 - accuracy: 0.4967 - 1s/epoch - 66ms/step
Epoch 13/30
19/19 - 1s - loss: 0.6932 - accuracy

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
29/29 - 8s - loss: 0.6937 - accuracy: 0.4789 - 8s/epoch - 271ms/step
Epoch 2/30
29/29 - 2s - loss: 0.6936 - accuracy: 0.4800 - 2s/epoch - 66ms/step
Epoch 3/30
29/29 - 2s - loss: 0.6942 - accuracy: 0.4812 - 2s/epoch - 66ms/step
Epoch 4/30
29/29 - 2s - loss: 0.6933 - accuracy: 0.4978 - 2s/epoch - 65ms/step
Epoch 5/30
29/29 - 2s - loss: 0.6936 - accuracy: 0.5044 - 2s/epoch - 65ms/step
Epoch 6/30
29/29 - 2s - loss: 0.6937 - accuracy: 0.5100 - 2s/epoch - 66ms/step
Epoch 7/30
29/29 - 2s - loss: 0.6925 - accuracy: 0.5188 - 2s/epoch - 65ms/step
Epoch 8/30
29/29 - 2s - loss: 0.6947 - accuracy: 0.4834 - 2s/epoch - 64ms/step
Epoch 9/30
29/29 - 2s - loss: 0.6936 - accuracy: 0.4667 - 2s/epoch - 64ms/step
Epoch 10/30
29/29 - 2s - loss: 0.6932 - accuracy: 0.4911 - 2s/epoch - 64ms/step
Epoch 11/30
29/29 - 2s - loss: 0.6930 - accuracy: 0.5000 - 2s/epoch - 65ms/step
Epoch 12/30
29/29 - 2s - loss: 0.6933 - accuracy: 0.4778 - 2s/epoch - 64ms/step
Epoch 13/30
29/29 - 2s - loss: 0.6932 - accuracy

In [11]:
results_df_lstm = pd.DataFrame(grid_result_lstm.cv_results_)
results_df_lstm[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.511096,8
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.504430,13
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.508889,10
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.505537,11
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.543215,2
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.514433,6
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.529904,4
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.539889,3
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.577519,1
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.524441,5


In [12]:
print("Best Parameters (LSTM): ", grid_result_lstm.best_params_)
print("Best Accuracy (LSTM): ", grid_result_lstm.best_score_)

Best Parameters (LSTM):  {'learning_rate': 0.001, 'optimizer': 'Adadelta'}
Best Accuracy (LSTM):  0.5775193798449613


In [19]:
best_lstm_model = grid_result_lstm.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_lstm_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


8/8 - 0s - 203ms/epoch - 25ms/step
Test Accuracy: 0.5132743362831859
Precision: 0.5067264573991032
Recall: 1.0
F1-score: 0.6726190476190477


## Bi-LSTM

In [14]:
def create_bi_lstm_model(optimizer='adam', learning_rate=0.001):
    bi_lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Bidirectional(LSTM(64)),
        Dense(1, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'
    
    bi_lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return bi_lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
bi_lstm_classifier = KerasClassifier(build_fn=create_bi_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search = GridSearchCV(estimator=bi_lstm_classifier, param_grid=param_grid, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result = grid_search.fit(X_train_padded, y_train)



C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 8s - loss: 0.6932 - accuracy: 0.4592 - 8s/epoch - 409ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 555ms/epoch - 29ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 547ms/epoch - 29ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4676 - 547ms/epoch - 29ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4759 - 545ms/epoch - 29ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4842 - 542ms/epoch - 29ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.5008 - 538ms/epoch - 28ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.5008 - 544ms/epoch - 29ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.5008 - 539ms/epoch - 28ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.5008 - 544ms/epoch - 29ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4576 - 546ms/epoch - 29ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.5008 - 545ms/epoch - 29ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 8s - loss: 0.6932 - accuracy: 0.4975 - 8s/epoch - 401ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 548ms/epoch - 29ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 576ms/epoch - 30ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 555ms/epoch - 29ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 547ms/epoch - 29ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 548ms/epoch - 29ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 549ms/epoch - 29ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 549ms/epoch - 29ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 545ms/epoch - 29ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 544ms/epoch - 29ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4792 - 545ms/epoch - 29ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 551ms/epoch - 29ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
19/19 - 6s - loss: 0.6932 - accuracy: 0.4992 - 6s/epoch - 323ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4526 - 521ms/epoch - 27ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 520ms/epoch - 27ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 525ms/epoch - 28ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4709 - 526ms/epoch - 28ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4925 - 525ms/epoch - 28ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 524ms/epoch - 28ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4775 - 520ms/epoch - 27ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 539ms/epoch - 28ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4992 - 535ms/epoch - 28ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.4809 - 536ms/epoch - 28ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6932 - accuracy: 0.5008 - 530ms/epoch - 28ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 6s - loss: 0.6933 - accuracy: 0.4950 - 6s/epoch - 322ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6904 - accuracy: 0.5449 - 543ms/epoch - 29ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6875 - accuracy: 0.5050 - 542ms/epoch - 29ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6802 - accuracy: 0.5748 - 544ms/epoch - 29ms/step
Epoch 5/30
19/19 - 1s - loss: 0.6637 - accuracy: 0.5930 - 545ms/epoch - 29ms/step
Epoch 6/30
19/19 - 1s - loss: 0.6690 - accuracy: 0.6047 - 541ms/epoch - 28ms/step
Epoch 7/30
19/19 - 1s - loss: 0.6547 - accuracy: 0.6960 - 566ms/epoch - 30ms/step
Epoch 8/30
19/19 - 1s - loss: 0.6382 - accuracy: 0.6711 - 545ms/epoch - 29ms/step
Epoch 9/30
19/19 - 1s - loss: 0.6378 - accuracy: 0.6661 - 536ms/epoch - 28ms/step
Epoch 10/30
19/19 - 1s - loss: 0.6279 - accuracy: 0.6811 - 543ms/epoch - 29ms/step
Epoch 11/30
19/19 - 1s - loss: 0.6214 - accuracy: 0.6844 - 550ms/epoch - 29ms/step
Epoch 12/30
19/19 - 1s - loss: 0.6095 - accuracy: 0.7226 - 552ms/epoch - 29ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
29/29 - 8s - loss: 0.6932 - accuracy: 0.4956 - 8s/epoch - 263ms/step
Epoch 2/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.5000 - 832ms/epoch - 29ms/step
Epoch 3/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.4889 - 809ms/epoch - 28ms/step
Epoch 4/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.5044 - 853ms/epoch - 29ms/step
Epoch 5/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.5000 - 803ms/epoch - 28ms/step
Epoch 6/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.4845 - 813ms/epoch - 28ms/step
Epoch 7/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.5000 - 796ms/epoch - 27ms/step
Epoch 8/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.5000 - 796ms/epoch - 27ms/step
Epoch 9/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.5000 - 782ms/epoch - 27ms/step
Epoch 10/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.4712 - 819ms/epoch - 28ms/step
Epoch 11/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.4911 - 818ms/epoch - 28ms/step
Epoch 12/30
29/29 - 1s - loss: 0.6932 - accuracy: 0.4823 - 870ms/epoch - 30ms/step
Epoch 13/30
29/

In [15]:
results_df = pd.DataFrame(grid_result.cv_results_)
results_df[['params', 'mean_test_score', 'rank_test_score']]
# Print the best parameters and corresponding accuracy

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.498893,13
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.590808,9
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.579734,11
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.498893,13
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.677486,3
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.590004,10
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.498893,13
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.578627,12
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.681912,2
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.596346,5


In [16]:
print("\n",grid_result)
print("\n")
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)



 GridSearchCV(cv=3,
             estimator=KerasClassifier(build_fn=<function create_bi_lstm_model at 0x0000026C7F79E4D0>, epochs=30, learning_rate=0.01, verbose=2),
             param_grid={'learning_rate': [0.01, 0.001, 0.0001],
                         'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta',
                                       'Nadam']},
             scoring='accuracy')


Best Parameters:  {'learning_rate': 0.0001, 'optimizer': 'Adagrad'}
Best Accuracy:  0.7738390550018458


In [18]:
best_model = grid_result.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)

from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

8/8 - 0s - 125ms/epoch - 16ms/step
Test Accuracy: 0.5
Precision: 0.5
Recall: 1.0
F1-score: 0.6666666666666666
